In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.neural_network import MLPClassifier

df_train = pd.read_csv("training.csv")
text_data = np.array(df_train['article_words'])
count = CountVectorizer()
bag_of_words = count.fit_transform(text_data)
X = bag_of_words
topic = {'IRRELEVANT':0, 'ARTS CULTURE ENTERTAINMENT':1, 'BIOGRAPHIES PERSONALITIES PEOPLE':2, 'DEFENCE':3, 'DOMESTIC MARKETS':4, 'FOREX MARKETS':5, 'HEALTH':6, 'MONEY MARKETS':7, 'SCIENCE AND TECHNOLOGY':8, 'SHARE LISTINGS':9, 'SPORTS':10}
Y = []
for i in range(len(df_train)):
    Y.append(topic[df_train.at[i, 'topic']])
    
X_train = X[:9001]
X_dev = X[9001:]
Y_train = Y[:9001]
Y_dev = Y[9001:]

In [2]:
print('MLPClassifier')
mlp = MLPClassifier()
model = mlp.fit(X, Y) 
predicted_train = model.predict(X_train)
print('Training data accuracy score is:',accuracy_score(Y_train, predicted_train))
predicted_dev = model.predict(X_dev)
print('Development data accuracy score is:',accuracy_score(Y_dev, predicted_dev))
df_test = pd.read_csv('test.csv') 
test_data = np.array(df_test['article_words'])
topic = {'IRRELEVANT':0, 'ARTS CULTURE ENTERTAINMENT':1, 'BIOGRAPHIES PERSONALITIES PEOPLE':2, 'DEFENCE':3, 'DOMESTIC MARKETS':4, 'FOREX MARKETS':5, 'HEALTH':6, 'MONEY MARKETS':7, 'SCIENCE AND TECHNOLOGY':8, 'SHARE LISTINGS':9, 'SPORTS':10}
Y_test = []
for i in range(len(df_test)):
    Y_test.append(topic[df_test.at[i, 'topic']])
result = []
prob = []
for i in range(len(df_test)):
    tmp = count.transform([test_data[i]])
    prob.append(model.predict_proba(tmp)[0])
    result.append(int(model.predict(tmp)[0]))
print('Test data accuracy score is:',accuracy_score(Y_test, result))

MLPClassifier
Training data accuracy score is: 0.9886679257860238
Development data accuracy score is: 0.9859719438877755
Test data accuracy score is: 0.746


In [3]:
#统计每个article对应的所有probability
probability = np.array(prob).T
probability.shape

mi = dict(zip(topic.values(), topic.keys()))
for i in range(len(result)):
    result[i] = mi[result[i]]
    
data = {'Article':pd.Series(df_test['article_number']),
       'ARTS CULTURE ENTERTAINMENT':pd.Series(probability[1]),
        'BIOGRAPHIES PERSONALITIES PEOPLE':pd.Series(probability[2]),
        'DEFENCE':pd.Series(probability[3]),
       'DOMESTIC MARKETS':pd.Series(probability[4]),
       'FOREX MARKETS':pd.Series(probability[5]),
       'HEALTH':pd.Series(probability[6]),
       'MONEY MARKETS':pd.Series(probability[7]),
       'SCIENCE AND TECHNOLOGY':pd.Series(probability[8]),
       'SHARE LISTINGS':pd.Series(probability[9]),
        'SPORTS':pd.Series(probability[10]),
       }
df = pd.DataFrame(data)

for i in range(1,len(df.columns)):
    tmp = df.sort_values(by=[df.columns[i]]).head(10)
    print(df.columns[i], list(tmp['Article']))

ARTS CULTURE ENTERTAINMENT [9681, 9515, 9519, 9923, 9857, 9905, 9781, 9914, 9968, 9760]
BIOGRAPHIES PERSONALITIES PEOPLE [9681, 9515, 9923, 9519, 9857, 9914, 9781, 9997, 9882, 9905]
DEFENCE [9681, 9515, 9519, 9857, 9923, 9774, 9752, 9760, 9596, 9787]
DOMESTIC MARKETS [9681, 9515, 9857, 9760, 9752, 9848, 9905, 9596, 9519, 9922]
FOREX MARKETS [9681, 9515, 9923, 9519, 9857, 9997, 9609, 9576, 9781, 9914]
HEALTH [9681, 9515, 9923, 9857, 9519, 9997, 9752, 9760, 9787, 9905]
MONEY MARKETS [9681, 9515, 9923, 9519, 9857, 9774, 9905, 9752, 9781, 9914]
SCIENCE AND TECHNOLOGY [9681, 9515, 9923, 9519, 9857, 9774, 9752, 9997, 9787, 9905]
SHARE LISTINGS [9681, 9515, 9923, 9857, 9519, 9576, 9760, 9597, 9997, 9774]
SPORTS [9681, 9515, 9923, 9519, 9914, 9968, 9905, 9781, 9775, 9750]


In [4]:
# 统计predict_result对应的probability
score = []
for i in prob:
    score.append(max(i))

r = {'Article':pd.Series(df_test['article_number']),
     'Result':pd.Series(np.array(result)),
     'Probability': pd.Series(np.array(score))       
       }
predict_prob = pd.DataFrame(r)

tp = list(topic.keys())
for i in range(1, len(tp)):
    tmp = predict_prob[predict_prob['Result'] == tp[i]].sort_values(by=['Probability'],ascending=False).head(10)
    print(tp[i], list(tmp['Article']))

ARTS CULTURE ENTERTAINMENT [9830, 9703, 9604, 9952, 9789, 9834]
BIOGRAPHIES PERSONALITIES PEOPLE [9695, 9854, 9988, 9940, 9933]
DEFENCE [9616, 9770, 9576, 9559, 9904, 9706, 9773, 9842, 9612, 9987]
DOMESTIC MARKETS [9994, 9796, 9989, 9566, 9617, 9910]
FOREX MARKETS [9693, 9984, 9901, 9986, 9894, 9729, 9772, 9902, 9748, 9530]
HEALTH [9609, 9807, 9833, 9661, 9873, 9978, 9929, 9735, 9887, 9947]
MONEY MARKETS [9938, 9751, 9835, 9871, 9516, 9503, 9755, 9953, 9792, 9971]
SCIENCE AND TECHNOLOGY [9982, 9621]
SHARE LISTINGS [9666, 9601, 9518, 9667]
SPORTS [9760, 9848, 9774, 9597, 9596, 9752, 9857, 9791, 9568, 9922]


In [5]:
print('SGDClassifier')
from sklearn.calibration import CalibratedClassifierCV
clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3, loss='hinge',class_weight='balanced')
clf = clf.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
predicted_train = model.predict(X_train)
print('Training data accuracy score is:',accuracy_score(Y_train, predicted_train))
predicted_dev = model.predict(X_dev)
print('Development data accuracy score is:',accuracy_score(Y_dev, predicted_dev))
df_test = pd.read_csv('test.csv')
test_data = np.array(df_test['article_words'])
topic = {'IRRELEVANT':0, 'ARTS CULTURE ENTERTAINMENT':1, 'BIOGRAPHIES PERSONALITIES PEOPLE':2, 'DEFENCE':3, 'DOMESTIC MARKETS':4, 'FOREX MARKETS':5, 'HEALTH':6, 'MONEY MARKETS':7, 'SCIENCE AND TECHNOLOGY':8, 'SHARE LISTINGS':9, 'SPORTS':10}
Y_test = []
for i in range(len(df_test)):
    Y_test.append(topic[df_test.at[i, 'topic']])
result = []
prob = []
for i in range(len(df_test)):
    tmp = count.transform([test_data[i]])
    prob.append(model.predict_proba(tmp)[0])
    result.append(int(model.predict(tmp)[0]))
print('Test data accuracy score is:',accuracy_score(Y_test, result))

SGDClassifier


/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/calibration.py:453: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/calibration.py:453: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/calibration.py:453: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
/Users/mark/o

Training data accuracy score is: 0.9400066659260082
Development data accuracy score is: 0.7194388777555111
Test data accuracy score is: 0.728


In [6]:
print('SVM')
clf = SVC()
model = clf.fit(X_train, Y_train)
predicted_train = model.predict(X_train)
print('Training data accuracy score is:',accuracy_score(Y_train, predicted_train))
predicted_dev = model.predict(X_dev)
print('Development data accuracy score is:',accuracy_score(Y_dev, predicted_dev))
df_test = pd.read_csv('test.csv')
test_data = np.array(df_test['article_words'])
topic = {'IRRELEVANT':0, 'ARTS CULTURE ENTERTAINMENT':1, 'BIOGRAPHIES PERSONALITIES PEOPLE':2, 'DEFENCE':3, 'DOMESTIC MARKETS':4, 'FOREX MARKETS':5, 'HEALTH':6, 'MONEY MARKETS':7, 'SCIENCE AND TECHNOLOGY':8, 'SHARE LISTINGS':9, 'SPORTS':10}
Y_test = []
for i in range(len(df_test)):
    Y_test.append(topic[df_test.at[i, 'topic']])
result = []
for i in range(len(df_test)):
    tmp = count.transform([test_data[i]])
    result.append(int(model.predict(tmp)[0]))
print('Test data accuracy score is:',accuracy_score(Y_test, result))

SVM


/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Training data accuracy score is: 0.5684923897344739
Development data accuracy score is: 0.5871743486973948
Test data accuracy score is: 0.604


In [7]:
model = XGBClassifier()
model.fit(X_train, Y_train)
predicted_train = model.predict(X_train)
print('Training data accuracy score is:',accuracy_score(Y_train, predicted_train))
predicted_dev = model.predict(X_dev)
print('Development data accuracy score is:',accuracy_score(Y_dev, predicted_dev))
df_test = pd.read_csv('test.csv')
test_data = np.array(df_test['article_words'])
topic = {'IRRELEVANT':0, 'ARTS CULTURE ENTERTAINMENT':1, 'BIOGRAPHIES PERSONALITIES PEOPLE':2, 'DEFENCE':3, 'DOMESTIC MARKETS':4, 'FOREX MARKETS':5, 'HEALTH':6, 'MONEY MARKETS':7, 'SCIENCE AND TECHNOLOGY':8, 'SHARE LISTINGS':9, 'SPORTS':10}
Y_test = []
for i in range(len(df_test)):
    Y_test.append(topic[df_test.at[i, 'topic']])
result = []
for i in range(len(df_test)):
    tmp = count.transform([test_data[i]])
    result.append(int(model.predict(tmp)[0]))
print('Test data accuracy score is:',accuracy_score(Y_test, result))

Training data accuracy score is: 0.847683590712143
Development data accuracy score is: 0.7675350701402806
Test data accuracy score is: 0.736


In [8]:
print('RandomForestClassifier')
clf = RandomForestClassifier()
model = clf.fit(X, Y)
predicted_train = model.predict(X_train)
print('Training data accuracy score is:',accuracy_score(Y_train, predicted_train))
predicted_dev = model.predict(X_dev)
print('Development data accuracy score is:',accuracy_score(Y_dev, predicted_dev))

df_test = pd.read_csv('test.csv') 
test_data = np.array(df_test['article_words'])
topic = {'IRRELEVANT':0, 'ARTS CULTURE ENTERTAINMENT':1, 'BIOGRAPHIES PERSONALITIES PEOPLE':2, 'DEFENCE':3, 'DOMESTIC MARKETS':4, 'FOREX MARKETS':5, 'HEALTH':6, 'MONEY MARKETS':7, 'SCIENCE AND TECHNOLOGY':8, 'SHARE LISTINGS':9, 'SPORTS':10}
Y_test = []
for i in range(len(df_test)):
    Y_test.append(topic[df_test.at[i, 'topic']])
result = []
prob = []
for i in range(len(df_test)):
    tmp = count.transform([test_data[i]])
    prob.append(model.predict_proba(tmp)[0])
    result.append(int(model.predict(tmp)[0]))
print('Test data accuracy score is:',accuracy_score(Y_test, result))


RandomForestClassifier


/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training data accuracy score is: 0.9793356293745139
Development data accuracy score is: 0.9759519038076152
Test data accuracy score is: 0.73


In [9]:
print('MultinomialNB')
clf = MultinomialNB()
model = clf.fit(X_train, Y_train)
predicted_train = model.predict(X_train)
print('Training data accuracy score is:',accuracy_score(Y_train, predicted_train))
predicted_dev = model.predict(X_dev)
print('Development data accuracy score is:',accuracy_score(Y_dev, predicted_dev))
df_test = pd.read_csv('test.csv')
test_data = np.array(df_test['article_words'])
topic = {'IRRELEVANT':0, 'ARTS CULTURE ENTERTAINMENT':1, 'BIOGRAPHIES PERSONALITIES PEOPLE':2, 'DEFENCE':3, 'DOMESTIC MARKETS':4, 'FOREX MARKETS':5, 'HEALTH':6, 'MONEY MARKETS':7, 'SCIENCE AND TECHNOLOGY':8, 'SHARE LISTINGS':9, 'SPORTS':10}
Y_test = []
for i in range(len(df_test)):
    Y_test.append(topic[df_test.at[i, 'topic']])
result = []
for i in range(len(df_test)):
    tmp = count.transform([test_data[i]])
    result.append(int(model.predict(tmp)[0]))
print('Test data accuracy score is:',accuracy_score(Y_test, result))

MultinomialNB
Training data accuracy score is: 0.8287968003555161
Development data accuracy score is: 0.7314629258517034
Test data accuracy score is: 0.722


In [10]:
print('BernoulliNB')
clf = BernoulliNB()
model = clf.fit(X_train, Y_train)
predicted_train = model.predict(X_train)
print('Training data accuracy score is:',accuracy_score(Y_train, predicted_train))
predicted_dev = model.predict(X_dev)
print('Development data accuracy score is:',accuracy_score(Y_dev, predicted_dev))
df_test = pd.read_csv('test.csv')
test_data = np.array(df_test['article_words'])
topic = {'IRRELEVANT':0, 'ARTS CULTURE ENTERTAINMENT':1, 'BIOGRAPHIES PERSONALITIES PEOPLE':2, 'DEFENCE':3, 'DOMESTIC MARKETS':4, 'FOREX MARKETS':5, 'HEALTH':6, 'MONEY MARKETS':7, 'SCIENCE AND TECHNOLOGY':8, 'SHARE LISTINGS':9, 'SPORTS':10}
Y_test = []
for i in range(len(df_test)):
    Y_test.append(topic[df_test.at[i, 'topic']])
result = []
for i in range(len(df_test)):
    tmp = count.transform([test_data[i]])
    result.append(int(model.predict(tmp)[0]))
print('Test data accuracy score is:',accuracy_score(Y_test, result))

BernoulliNB
Training data accuracy score is: 0.732140873236307
Development data accuracy score is: 0.7194388777555111
Test data accuracy score is: 0.674


In [11]:
print('DecisionTreeClassifier')
clf = tree.DecisionTreeClassifier()
model = clf.fit(X_train, Y_train)
predicted_train = model.predict(X_train)
print('Training data accuracy score is:',accuracy_score(Y_train, predicted_train))
predicted_dev = model.predict(X_dev)
print('Development data accuracy score is:',accuracy_score(Y_dev, predicted_dev))
df_test = pd.read_csv('test.csv')
test_data = np.array(df_test['article_words'])
topic = {'IRRELEVANT':0, 'ARTS CULTURE ENTERTAINMENT':1, 'BIOGRAPHIES PERSONALITIES PEOPLE':2, 'DEFENCE':3, 'DOMESTIC MARKETS':4, 'FOREX MARKETS':5, 'HEALTH':6, 'MONEY MARKETS':7, 'SCIENCE AND TECHNOLOGY':8, 'SHARE LISTINGS':9, 'SPORTS':10}
Y_test = []
for i in range(len(df_test)):
    Y_test.append(topic[df_test.at[i, 'topic']])
result = []
for i in range(len(df_test)):
    tmp = count.transform([test_data[i]])
    result.append(int(model.predict(tmp)[0]))
print('Test data accuracy score is:',accuracy_score(Y_test, result))

DecisionTreeClassifier
Training data accuracy score is: 0.989667814687257
Development data accuracy score is: 0.6973947895791583
Test data accuracy score is: 0.688
